In [0]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Dense, Activation
from keras.layers import LSTM, Bidirectional, Input
from keras.layers import concatenate,merge
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

finaldf=pd.read_csv('google_nq_final_processed.csv')

Using TensorFlow backend.


In [0]:
import re
def removeTags(sent):
    s = sent.strip().split()
    
    stop_words = ['<P>', '</P>', '<Table>', '</Table>', '<Tr>', '</Tr>', '<Ul>', '<Ol>', '<Dl>', '</Ul>', '</Ol>', 
             '</Dl>', '<Li>', '<Dd>', '<Dt>', '</Li>', '</Dd>', '</Dt>', '<H1>', '</H1>', '<H2>', '</H2>', '<H3>', '</H3>',
            'wikipedia', '</Td>', '<Th>', '<H4>', '</H4>', '</Th>', '<Td>', 'Jump up', 'Jump to']
    s = [w for w in s if w not in stop_words]
    
    s = " ".join(s)
    # replace unnecessary characters with space
    s = re.sub(r"[^A-Za-z0-9]", " ", str(s).lower())    
    
    return s

In [0]:
new_doc=[]
new_ans=[]
for i in range(len(finaldf)):
    new_doc.append(removeTags(finaldf.iloc[i]['document_text']))
    new_ans.append(removeTags(finaldf.iloc[i]['answer']))
    
finaldf['document']=new_doc
finaldf['ans']=new_ans
finaldf=finaldf.drop(columns=['document_text','answer'])
len(finaldf)

33347

In [0]:
count_df=finaldf['document'].apply(lambda x: len(x))
df_small = finaldf[count_df <= 3000]
df_small.reset_index(inplace=True, drop=True)
len(df_small)

1655

In [0]:
train=df_small[:1400]
test=df_small[1400:]
X_train,y_train=train[['document','question_text']],train[['start','end']]
X_test,y_test=test[['document','question_text']],test[['start','end']]


In [0]:
tokenizer = Tokenizer() #normally its somethinglike 40800
tokenizer.fit_on_texts(X_train['document'])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
lenght_list=[]
for l in X_train.question_text:
    lenght_list.append(len(l.split(' ')))
max_ques= np.max(lenght_list)
max_ques

lenght_list=[]
for l in X_train.document:
    lenght_list.append(len(l.split(' ')))
max_doc= np.max(lenght_list)
max_doc


1066

In [0]:
def loadGloveModel(gloveFile):
    f = open(gloveFile,'r')
    embedding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs 
    f.close()
    print('Found %s word vectors.' % len(embedding_index))
    return embedding_index

print('Preparing embedding matrix.')
embeddings_index = loadGloveModel('glove.6B.100d.txt')
nb_words = vocab_size
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = max_doc

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
print('Embedding matrix completed.')

Preparing embedding matrix.
Found 400000 word vectors.
Embedding matrix completed.


In [0]:
''' 
To perform vectorization of text records, 
these two functions are taken from 'https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras' with necessary modifications.

Vectorize the words to their respective index and pad context to max context length and question to max question length.
Answers vectors are padded to the max context length as well.
'''
def vectorizeData(xContext, xQuestion, xAnswerBeing, xAnswerEnd, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        y_Begin =  np.zeros(len(xContext[i]))
        y_Begin[xAnswerBeing[i]] = 1
        y_End = np.zeros(len(xContext[i]))
        y_End[xAnswerEnd[i]] = 1
        X.append(x)
        Xq.append(xq)
        YBegin.append(y_Begin)
        YEnd.append(y_End)
    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post'), pad_sequences(YBegin, maxlen=context_maxlen, padding='post'), pad_sequences(YEnd, maxlen=context_maxlen, padding='post')

def vectorizeValData(xContext, xQuestion, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        X.append(x)
        Xq.append(xq)

    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post')


In [0]:
tX, tXq, tYBegin, tYEnd = vectorizeData(X_train['document'],X_train['question_text'],y_train['start'],y_train['end'],tokenizer.word_index,max_doc,max_ques)
tX.shape, tXq.shape, tYBegin.shape, tYEnd.shape 

((1400, 1066), (1400, 18), (1400, 1066), (1400, 1066))

In [0]:
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, RepeatVector, Activation, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU, concatenate, Recurrent, InputSpec
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Attention
import re
import gcsfs
from keras import initializers

'''
To calculate Bahdanau Attention, the given class is taken from 
https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras/blob/master/rnnlayer.py , as it was already implemented and available.
'''
# Bahdanau Attention 
class Attention(Recurrent):
    def __init__(self, units, h_dim,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal',
                 **kwargs):
        self.units = units
        self.h_dim = h_dim
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        self.input_dim = input_shape[2] - self.h_dim

        if self.stateful:
            self.reset_states()
        else:
            self.states = [None]

        self.Wa = self.add_weight('{}_Wa'.format(self.name), (self.units, self.units),
                                  initializer=self.recurrent_initializer)
        self.Ua = self.add_weight('{}_Ua'.format(self.name), (self.h_dim, self.units),
                                  initializer=self.recurrent_initializer)
        self.Va = self.add_weight('{}_Va'.format(self.name),(self.units,1),
                                  initializer=self.recurrent_initializer)
        self.Wzr = self.add_weight('{}_Wzr'.format(self.name), (self.input_dim, 2 * self.units),
                                 initializer=self.recurrent_initializer)
        self.Uzr = self.add_weight('{}_Wzr'.format(self.name), (self.units, 2 * self.units),
                                 initializer=self.recurrent_initializer)
        self.Czr = self.add_weight('{}_Czr'.format(self.name), (self.h_dim, 2 * self.units),
                                   initializer=self.recurrent_initializer)
        self.W = self.add_weight('{}_W'.format(self.name), (self.input_dim, self.units),
                                 initializer=self.recurrent_initializer)
        self.U = self.add_weight('{}_U'.format(self.name), (self.units, self.units),
                                 initializer=self.recurrent_initializer)
        self.C = self.add_weight('{}_C'.format(self.name), (self.h_dim, self.units),
                                 initializer=self.recurrent_initializer)
        self.built = True

    def reset_states(self):
        assert self.stateful, 'Layer must be stateful.'
        input_shape = self.input_spec[0].shape
        if not input_shape[0]:
            raise ValueError('If a RNN is stateful, a complete '
                             'input_shape must be provided '
                             '(including batch size).')
        if hasattr(self, 'states'):
            K.set_value(self.states[0],
                        np.zeros((input_shape[0], self.units)))
        else:
            self.states = [K.zeros((input_shape[0], self.units))]

    def preprocess_input(self, inputs, training=None):
        return inputs

    def step(self, inputs, states):
        h_tm1 = states[0]  # previous memory
        h_tm1a = K.dot(h_tm1, self.Wa)
        eij = K.dot(K.tanh(h_tm1a + K.dot(inputs[:, :self.h_dim], self.Ua)), self.Va)
        eijs = K.repeat_elements(eij, self.h_dim, axis=1)

        cisum = eijs*inputs[:, :self.h_dim]
        
        zr = K.sigmoid(K.dot(inputs[:, self.h_dim:], self.Wzr) + K.dot(h_tm1, self.Uzr) + K.dot(cisum, self.Czr))
        zi = zr[:, :self.units]
        ri = zr[:, self.units: 2 * self.units]
        si_ = K.tanh(K.dot(inputs[:, self.h_dim:], self.W) + K.dot(ri*h_tm1, self.U) + K.dot(cisum, self.C))
        si = (1-zi) * h_tm1 + zi * si_
        return si, [si] #h_tm1, [h_tm1]

    def get_constants(self, inputs, training=None):
        constants = []
        constants.append([K.cast_to_floatx(1.) for _ in range(3)])
        return constants

    def get_config(self):
        config = {'units': self.units,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer)}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
question_input = Input(shape=(max_ques,), dtype='int32', name='question_input')
context_input = Input(shape=(max_doc,), dtype='int32', name='context_input')
questionEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=max_ques, trainable=False)(question_input)  
contextEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=max_doc, trainable=False)(context_input) 

In [0]:
Q_h = Bidirectional(LSTM(100, return_sequences=True))(questionEmbd)
C_h = Bidirectional(LSTM(100, return_sequences=True))(contextEmbd)

merge1 = concatenate([Q_h, C_h], axis=1)
atten_1 = Attention(100, 100, return_sequences=True)(merge1)
dropout_1 = Dropout(0.25)(atten_1)

QC_bilstm = Bidirectional(LSTM(100, return_sequences=True))(dropout_1)
merge2 = concatenate([QC_bilstm, C_h], axis=1)
dropout_2 = Dropout(0.5)(merge2)

start_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)
end_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)

In [0]:
start_token_dense = LSTM(max_doc, activation='softmax')(start_bilstm)

end_token_dense = LSTM(max_doc, activation='softmax')(end_bilstm)

model = Model(input=[context_input, question_input], output=[start_token_dense, end_token_dense])
adam = optimizers.Adam(lr=0.003)
model.compile(optimizer=adam, loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_input (InputLayer)     (None, 18)           0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      (None, 1066)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 18, 100)      3172000     question_input[0][0]             
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1066, 100)    3172000     context_input[0][0]              
____________________________________________________________________________________________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`


In [0]:

checkpoint2 = ModelCheckpoint('bilstm_with_atten_model.h5', monitor='val_dense_1_loss', verbose=1, save_best_only=True, mode='min', period=2)
checkpoint3 = EarlyStopping(monitor='val_dense_1_loss', min_delta=0.01, patience=3, verbose=1, mode='auto')
checkpoint4 = EarlyStopping(monitor='val_dense_2_loss', min_delta=0.01, patience=3, verbose=1, mode='auto')
callbacks_list = [checkpoint2, checkpoint3, checkpoint4] 
EPOCHS = 10
model.fit([tX, tXq], [tYBegin,tYEnd], epochs=EPOCHS, batch_size=64, shuffle=True, 
          validation_split=0.2, \
          callbacks=callbacks_list)


Train on 1120 samples, validate on 280 samples
Epoch 1/10
1120/1120 [==============================] - 125s 111ms/step - loss: 0.2801 - dense_1_loss: 0.1437 - dense_2_loss: 0.1537 - val_loss: 0.3390 - val_dense_1_loss: 0.1292 - val_dense_2_loss: 0.1674
Epoch 2/10
1120/1120 [==============================] - 121s 108ms/step - loss: 0.1680 - dense_1_loss: 0.0776 - dense_2_loss: 0.0857 - val_loss: 0.3445 - val_dense_1_loss: 0.1318 - val_dense_2_loss: 0.1696

Epoch 00002: val_dense_1_loss improved from inf to 0.13179, saving model to bilstm_with_atten_model.h5
Epoch 3/10
1120/1120 [==============================] - 122s 109ms/step - loss: 0.1732 - dense_1_loss: 0.0806 - dense_2_loss: 0.0878 - val_loss: 0.3450 - val_dense_1_loss: 0.1311 - val_dense_2_loss: 0.1708
Epoch 4/10
1120/1120 [==============================] - 122s 109ms/step - loss: 0.1692 - dense_1_loss: 0.0785 - dense_2_loss: 0.0860 - val_loss: 0.3444 - val_dense_1_loss: 0.1311 - val_dense_2_loss: 0.1703

Epoch 00004: val_dense_

In [0]:
# !pip install h5py
import h5py

model.save_weights('nq_with_attention_model_weights.h5')


In [0]:
vX, vXq, vStart, vEnd = vectorizeData(X_test['document'],X_test['question_text'],y_test['start'],y_test['end'],tokenizer.word_index,max_doc,max_ques)

predictions = model.predict([vX, vXq], batch_size=64)
print(predictions[0].shape, predictions[1].shape)
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
for i in range(predictions[0].shape[0]):
    ansBegin[i] = predictions[0][i, :].argmax()
    ansEnd[i] = predictions[1][i, :].argmax()
print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())

(255, 1066) (255, 1066)
19 336 33 1022


In [0]:
# F1-Score calculation
from sklearn.metrics import f1_score

start_f1 = f1_score(vStart ,ansBegin, average="weighted")
end_f1 = f1_score(vEnd ,ansEnd, average="weighted")

#F1
(start_f1 + end_f1) / 2

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


3.087849312953528e-05

- Because of F1-score~0, which is wrong, we didn't calculate EM score